In [ ]:
import requests, time, re
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
from dateutil import parser as dateparser

HEADERS = {"User-Agent": "Mozilla/5.0"}
OUT_DIR = Path("../data/raw/noticias_scrape")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Configura aquí los selectores del sitio que vayas a raspar
SELECTORS = {
    "item": "article",          # contenedor de cada noticia en el archivo
    "title": "a",               # dentro del item, selector del título
    "link_attr": "href",        # atributo con el link
    "date": "time",             # dentro del item, selector de fecha (si existe)
}

def get_html(url, sleep=1.0):
    r = requests.get(url, headers=HEADERS, timeout=20)
    time.sleep(sleep)
    if not r.ok:
        return None
    return r.text

def parse_date(text_or_html):
    if not text_or_html:
        return None
    try:
        return dateparser.parse(text_or_html, fuzzy=True)
    except:
        return None

def parse_article_page(link):
    """Plan B: si en la lista no viene fecha, abre la noticia y busca meta-date."""
    html = get_html(link, sleep=0.8)
    if not html:
        return None, None
    soup = BeautifulSoup(html, "html.parser")
    # meta comunes
    for sel, attr in [
        ('meta[property="article:published_time"]', "content"),
        ('meta[name="article:published_time"]', "content"),
        ('meta[itemprop="datePublished"]', "content"),
        ('time[itemprop="datePublished"]', None),
        ("time", None),
    ]:
        tag = soup.select_one(sel)
        if tag:
            val = tag.get(attr) if attr else tag.get_text(strip=True)
            dt = parse_date(val)
            if dt:
                # título básico
                title = soup.title.text.strip() if soup.title else None
                return title, dt
    # fallback: intenta fecha en el texto
    dt = parse_date(soup.get_text(" ", strip=True))
    title = soup.title.text.strip() if soup.title else None
    return title, dt

def scrape_archive_list(url):
    """Devuelve lista de (title, link, date_text) desde una página de archivo."""
    html = get_html(url)
    if not html:
        return []
    soup = BeautifulSoup(html, "html.parser")
    items = soup.select(SELECTORS["item"])
    rows = []
    for it in items:
        a = it.select_one(SELECTORS["title"])
        if not a:
            continue
        title = a.get_text(strip=True)
        link = a.get(SELECTORS["link_attr"])
        if link and link.startswith("/"):
            # ajusta si necesitas prefijo de dominio
            # link = "https://EL-DOMINIO.com" + link
            pass
        date_el = it.select_one(SELECTORS["date"])
        date_txt = date_el.get_text(strip=True) if date_el else None
        rows.append((title, link, date_txt))
    return rows

def build_archive_urls(year):
    """
    ADAPTA ESTO A CADA WEB:
    Devuelve una lista de URLs de archivo para un año concreto.
    Ejemplos:
      - f"https://medio.com/archivo/{year}/?page={p}"
      - f"https://medio.com/buscar?year={year}&page={p}"
    """
    urls = []
    for p in range(1, 11):   # ajusta nº de páginas
        urls.append(f"https://EJEMPLO.com/archivo/{year}/?page={p}")
    return urls

def scrape_year(site_name, year):
    """
    Raspa un año completo:
     1) recorre páginas de archivo
     2) intenta fecha en la lista; si no, abre la noticia
     3) guarda CSV
    """
    expected_cols = ["Date","Title","Link","ArchiveURL","Year_target","Site"]
    all_rows = []

    archive_urls = build_archive_urls(year)
    for url in archive_urls:
        print("Archivo:", url)
        triplets = scrape_archive_list(url)
        print(f"  -> items encontrados en lista: {len(triplets)}")
        for title, link, date_txt in triplets:
            dt = parse_date(date_txt)
            # si la lista no trae fecha, abre la noticia
            if not dt and link:
                title2, dt2 = parse_article_page(link)
                if title2 and len(title2) > 8:
                    title = title2
                if dt2:
                    dt = dt2
            all_rows.append({
                "Date": dt.isoformat() if dt else None,
                "Title": title,
                "Link": link,
                "ArchiveURL": url,
                "Year_target": year,
                "Site": site_name
            })

    # --- crear DF con columnas garantizadas ---
    if not all_rows:
        df = pd.DataFrame(columns=expected_cols)
        print(f"[{site_name} {year}] No se encontraron artículos con los selectores/URLs actuales.")
        return df

    df = pd.DataFrame(all_rows)
    for c in expected_cols:
        if c not in df.columns:
            df[c] = None

    # filtra por año objetivo si hay fecha
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df[df["Date"].dt.year.eq(year) | df["Date"].isna()].reset_index(drop=True)

    out = OUT_DIR / f"{site_name}_{year}.csv"
    df.to_csv(out, index=False, encoding="utf-8-sig")
    print(f"Guardado {out} ({len(df)} filas)")
    return df



In [5]:
# Ejemplo (DEBES adaptar dominio/selector reales del sitio que elijas)
def build_archive_urls(year):
    return [f"https://www.bbva.com/es/sala-de-prensa/{year}/page/{p}/" for p in range(1, 8)]

SELECTORS = {
    "item": "article",      # por ejemplo
    "title": "h2 a",
    "link_attr": "href",
    "date": "time"
}

df_bbva_2000 = scrape_year("BBVA", 2000)
# Repite para 2001..2005


Archivo: https://www.bbva.com/es/sala-de-prensa/2000/page/1/
Archivo: https://www.bbva.com/es/sala-de-prensa/2000/page/2/
Archivo: https://www.bbva.com/es/sala-de-prensa/2000/page/3/
Archivo: https://www.bbva.com/es/sala-de-prensa/2000/page/4/
Archivo: https://www.bbva.com/es/sala-de-prensa/2000/page/5/
Archivo: https://www.bbva.com/es/sala-de-prensa/2000/page/6/
Archivo: https://www.bbva.com/es/sala-de-prensa/2000/page/7/


KeyError: 'Date'